#Group the [AAC, ARG, AAC] Parsed reports


In [0]:
#Libraries
import pandas as pd
import unicodedata as ud
import re
import csv

#Paths
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'
FINAL_FILE_DF = DATASETS_FOLDER + 'DF_ReportsInColumns_v19.11.29.pkl'

In [0]:
#Take Files From GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Import FILES
dfAAC = pd.read_pickle( DATASETS_FOLDER + "DF_ReportsInColumns_AAC_v19.11.21a.pkl")
dfAAG = pd.read_pickle( DATASETS_FOLDER + "DF_ReportsInColumns_AAG_v19.10.24.pkl")
dfARG = pd.read_pickle( DATASETS_FOLDER + "DF_ReportsInColumns_ARG_v19.11.07.pkl")


In [0]:
#Import list of Files
dfAACFiles = pd.read_pickle( DATASETS_FOLDER + "DF_Links_Reports_AAC.pkl")
dfAAGFiles = pd.read_pickle( DATASETS_FOLDER + "DF_Links_Reports_AAG.pkl")
dfARGFiles = pd.read_pickle( DATASETS_FOLDER + "DF_Links_Reports_ARG.pkl")


In [0]:
#Some Info about Files
print('AAC - Files: ' + str(len(dfAACFiles)) + ' Files Parsed: ' + str(len(dfAAC.ReportNumber.unique().tolist())) )
print('AAG - Files: ' + str(len(dfAAGFiles)) + ' Files Parsed: ' + str(len(dfAAG.ReportNumber.unique().tolist())) )
print('ARG - Files: ' + str(len(dfARGFiles)) + ' Files Parsed: ' + str(len(dfARG.ReportNumber.unique().tolist())) )


AAC - Files: 4019 Files Parsed: 3901
AAG - Files: 847 Files Parsed: 798
ARG - Files: 1058 Files Parsed: 1005


In [0]:
#List of OS
lstReportNumbers=dfAAC.ReportNumber.unique().tolist()
lstReportNumbers.extend( dfAAG.ReportNumber.unique().tolist() )
lstReportNumbers.extend( dfARG.ReportNumber.unique().tolist() )

print('AAC OS: ' + str(len(dfAAC.ReportNumber.unique().tolist())))
print('AAG OS: ' + str(len(dfAAG.ReportNumber.unique().tolist())))
print('ARG OS: ' + str(len(dfARG.ReportNumber.unique().tolist())))
print('TOTAL : ' + str(len(lstReportNumbers)))

AAC OS: 3901
AAG OS: 798
ARG OS: 1005
TOTAL : 5704


In [0]:
#Export LST to .CSV
with open(DATASETS_FOLDER + 'ListOfReportNumbers.csv', 'w') as myfile:
  wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
  wr.writerow(lstReportNumbers)

In [0]:
#Clean the Files
#Delete ERROR LINES
def deleteErrorLines(df):
  df = df.drop_duplicates()
  df = df[ ~df.ReportNumber.str.contains('ERROR') ]
  df = df[ ~df.ConstNumber.str.contains('ERROR') ]
  df = df [ (df.ReportNumber.str.len() == 9) ].append( df [ (df.ReportNumber.str.len() == 6) ] ) 
  return df

#Show ERRORs
def showError(df):
  print( df[df.apply(lambda r: r.str.contains('ERROR', case=False).any(), axis=1)] )

In [0]:
# Clean the Files ERRORs
dfAAC_c = deleteErrorLines(dfAAC)
dfAAG_c = deleteErrorLines(dfAAG)
dfARG_c = deleteErrorLines(dfARG)

In [0]:
#Some Info about Files CLEAN
print('AAC - Files: ' + str(len(dfAACFiles)) + ' Files Parsed: ' + str(len(dfAAC.ReportNumber.unique().tolist())) + ' Files Parsed NO ERRORS: ' + str(len(dfAAC_c.ReportNumber.unique().tolist())) )
print('AAG - Files: ' + str(len(dfAAGFiles)) + ' Files Parsed: ' + str(len(dfAAG.ReportNumber.unique().tolist())) + ' Files Parsed NO ERRORS: ' + str(len(dfAAG_c.ReportNumber.unique().tolist())) )
print('ARG - Files: ' + str(len(dfARGFiles)) + ' Files Parsed: ' + str(len(dfARG.ReportNumber.unique().tolist())) + ' Files Parsed NO ERRORS: ' + str(len(dfARG_c.ReportNumber.unique().tolist())) )

AAC - Files: 4019 Files Parsed: 3901 Files Parsed NO ERRORS: 3834
AAG - Files: 847 Files Parsed: 798 Files Parsed NO ERRORS: 756
ARG - Files: 1058 Files Parsed: 1005 Files Parsed NO ERRORS: 981


In [0]:
#Count of Findings (Constat)
print( pd.pivot_table( dfAAC_c, values=['ConstNumber'],                          
                          columns=['ConstTYPE'],
                          aggfunc=len ))
print( pd.pivot_table( dfAAG_c, values=['ConstNumber'],                          
                          columns=['ConstTYPE'],
                          aggfunc=len ))
print( pd.pivot_table( dfARG_c, values=['ConstNumber'],                          
                          columns=['ConstTYPE'],
                          aggfunc=len ))
#Pivot Table
# print( pd.pivot_table( dfAAC_c, values=['ConstNumber'], 
#                           index=['ReportNumber'], 
#                           aggfunc=len ).sort_values(by='ConstNumber', ascending=False)
#  )


71660
ConstTYPE           conclusao  constatacao  informacao
ConstNumber  32426       3762        18469       17003
ConstTYPE         conclusao  constatacao  informacao
ConstNumber  299        731         2021        2197
ConstTYPE          conclusao  constatacao  informacao
ConstNumber  7262        855         2774        1300


In [0]:
#Write all data in just one DF
dfFinal = dfAAC_c
dfFinal = dfFinal.append(dfAAG_c)
dfFinal = dfFinal.append(dfARG_c)

print(len(dfAAC_c))
print(len(dfAAG_c))
print(len(dfARG_c))
print(len(dfFinal))

71660
5248
12191
89099


In [0]:
#Write DF to disk
dfFinal.to_pickle( FINAL_FILE_DF )